In [45]:
import csv
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statistics

import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest

In [46]:
df_mat_lv = pd.read_csv('MaternityLeave.csv')
print("The shape of the data: " + str(df_mat_lv.shape))
df_mat_lv.head()

The shape of the data: (972, 6)


,Name,Industry,Maternity Leave\nPaid (weeks),Maternity Leave\nUnpaid (weeks),Paternity Leave\nPaid (weeks),Paternity Leave\nUnpaid (weeks)
0,ClassPass,Wellness & Fitness,14.0,12.0,4.0,NaN
1,Equinox,Wellness & Fitness,6.0,12.0,NaN,NaN
2,Enterprise Holdings,Transportation: Rental,17.0,17.0,NaN,NaN
3,"Hertz Global Holdings, Inc.",Transportation: Rental,12.0,0.0,NaN,NaN
4,"Avis Budget Group, Inc.",Transportation: Rental,6.0,6.0,NaN,NaN


This dataframe contains information about the amount of maternity (and paternity) leave offered by 972 companies

In [47]:
df_mat_lv = df_mat_lv.drop(['Paternity Leave\nPaid (weeks)', 'Paternity Leave\nUnpaid (weeks)'], axis=1)
df_mat_lv.head()

,Name,Industry,Maternity Leave\nPaid (weeks),Maternity Leave\nUnpaid (weeks)
0,ClassPass,Wellness & Fitness,14.0,12.0
1,Equinox,Wellness & Fitness,6.0,12.0
2,Enterprise Holdings,Transportation: Rental,17.0,17.0
3,"Hertz Global Holdings, Inc.",Transportation: Rental,12.0,0.0
4,"Avis Budget Group, Inc.",Transportation: Rental,6.0,6.0


In [48]:
df_mat_lv.columns = ["Company Name", "Industry", "Maternity Leave (paid)", "Maternity Leave (unpaid)"]
df_mat_lv.head()

,Company Name,Industry,Maternity Leave (paid),Maternity Leave (unpaid)
0,ClassPass,Wellness & Fitness,14.0,12.0
1,Equinox,Wellness & Fitness,6.0,12.0
2,Enterprise Holdings,Transportation: Rental,17.0,17.0
3,"Hertz Global Holdings, Inc.",Transportation: Rental,12.0,0.0
4,"Avis Budget Group, Inc.",Transportation: Rental,6.0,6.0


In [49]:
df_mat_lv["Maternity Leave (unpaid)"] = df_mat_lv["Maternity Leave (unpaid)"].fillna(0.0)
df_mat_lv = df_mat_lv.dropna()
print("the shape of the dataframe is now: " + str(df_mat_lv.shape))

the shape of the dataframe is now: (965, 4)


In [50]:
df_mat_lv['Industry'].unique()

array(['Wellness & Fitness', 'Transportation: Rental', 'Wholesale',
       'Transportation: Rail', 'Transportation: Passenger Air',
       'Transportation: Freight & Logistics',
       'Transportation: Couriers & Delivery', 'Transportation: Bus',
       'Telecommunications', 'Technology: Software',
       'Technology: Security', 'Technology: Research',
       'Technology: Payments', 'Technology: Manufacturing',
       'Technology: Gaming', 'Technology: Financial Services',
       'Technology: Consumer Internet', 'Technology: B2B Tech Services',
       'Sports', 'Services: Waste Management', 'Services: Translation',
       'Services: Other', 'Services: Daycare', 'Services: Cleaning',
       'Retail: Supermarket Company',
       'Retail: Stationary & Office Supplies',
       'Retail: Shoes, Accessories and Apparel', 'Retail: Other',
       'Retail: Online', 'Retail: Jewelery',
       'Retail: General Merchandise',
       'Retail: Furniture & Home Improvement', 'Retail: Electronics',
    

In [87]:
df_industries_sum = pd.read_csv('cpsaat18sum.csv')
df_industries_sum.columns = ["Industry", "Total Unemployed", "Women", "White", "Black or African American", "Asian", "Hispanic or Latino"]
df_industries_sum

,Industry,Total Unemployed,Women,White,Black or African American,Asian,Hispanic or Latino
0,"Total, 16 years and over","157,538",47.0,77.7,12.3,6.5,17.6
1,"Agriculture, forestry, fishing, and hunting","2,425",26.2,93.0,2.2,1.7,27.5
2,"Mining, quarrying, and oil and gas extraction",750,15.8,86.7,5.1,3.7,20.1
3,Construction,"11,373",10.3,88.1,6.4,1.9,30.4
4,Manufacturing,"15,741",29.4,79.5,10.4,6.8,16.8
5,Wholesale and retail trade,"19,742",44.2,78.6,11.7,5.7,18.1
6,Transportation and utilities,"8,991",24.1,71.0,20.1,5.6,18.8
7,Information,"2,766",40.5,76.9,11.3,8.5,12.5
8,Financial activities,"10,765",52.6,79.6,10.5,7.5,12.9
9,Professional and business services,"19,606",41.3,77.4,10.0,9.6,16.0


In [69]:
df_occ_gender = pd.read_csv('inc_occ_gender.csv')
df_occ_gender.head()

,Occupation,All_workers,All_weekly,M_workers,M_weekly,F_workers,F_weekly
0,ALL OCCUPATIONS,109080,809,60746,895,48334,726
1,MANAGEMENT,12480,1351,7332,1486,5147,1139
2,Chief executives,1046,2041,763,2251,283,1836
3,General and operations managers,823,1260,621,1347,202,1002
4,Legislators,8,Na,5,Na,4,Na


In [71]:
df_occ_gender = df_occ_gender[df_occ_gender.All_weekly != 'Na']
df_occ_gender = df_occ_gender[df_occ_gender.M_weekly != 'Na']
df_occ_gender = df_occ_gender[df_occ_gender.F_weekly != 'Na']
df_occ_gender = df_occ_gender[df_occ_gender.M_workers != 'Na']
df_occ_gender = df_occ_gender[df_occ_gender.All_workers != 'Na']
df_occ_gender = df_occ_gender[df_occ_gender.F_workers != 'Na']

/Users/gaurikad/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [72]:
df_occ_gender

,Occupation,All_workers,All_weekly,M_workers,M_weekly,F_workers,F_weekly
0,ALL OCCUPATIONS,109080,809,60746,895,48334,726
1,MANAGEMENT,12480,1351,7332,1486,5147,1139
2,Chief executives,1046,2041,763,2251,283,1836
3,General and operations managers,823,1260,621,1347,202,1002
6,Marketing and sales managers,948,1462,570,1603,378,1258
...,...,...,...,...,...,...,...
521,TRANSPORTATION,6953,646,5998,679,955,494
527,Bus drivers,323,615,184,681,138,572
528,Driver/sales workers and truck drivers,2687,747,2582,751,105,632
550,"Laborers and freight, stock, and material move...",1433,526,1214,547,219,455


In [85]:
x1 = df_occ_gender.loc[df_occ_gender['Occupation'] == "MANAGEMENT"]
x2 = df_occ_gender.loc[df_occ_gender['Occupation'] == "BUSINESS"]
x3 = df_occ_gender.loc[df_occ_gender['Occupation'] == "COMPUTATIONAL"]
x4 = df_occ_gender.loc[df_occ_gender['Occupation'] == "ENGINEERING"]
x5 = df_occ_gender.loc[df_occ_gender['Occupation'] == "SCIENCE"]
x6 = df_occ_gender.loc[df_occ_gender['Occupation'] == "SOCIAL SERVICE"]
x7 = df_occ_gender.loc[df_occ_gender['Occupation'] == "LEGAL"]
x8 = df_occ_gender.loc[df_occ_gender['Occupation'] == "EDUCATION"]
x9 = df_occ_gender.loc[df_occ_gender['Occupation'] == "ARTS"]
x10 = df_occ_gender.loc[df_occ_gender['Occupation'] == "HEALTHCARE PROFESSIONAL"]
x11 = df_occ_gender.loc[df_occ_gender['Occupation'] == "CULLINARY"]
x12 = df_occ_gender.loc[df_occ_gender['Occupation'] == "SERVICE"]
x13 = df_occ_gender.loc[df_occ_gender['Occupation'] == "SALES"]
x14 = df_occ_gender.loc[df_occ_gender['Occupation'] == "OFFICE"]
x15 = df_occ_gender.loc[df_occ_gender['Occupation'] == "AGRICULTIRAL"]
x16 = df_occ_gender.loc[df_occ_gender['Occupation'] == "CONSTRUCTION"]
x17 = df_occ_gender.loc[df_occ_gender['Occupation'] == "MAINTAINANCE"]
x18 = df_occ_gender.loc[df_occ_gender['Occupation'] == "PRODUCTION"]
x19 = df_occ_gender.loc[df_occ_gender['Occupation'] == "TRANSPORTATION"]

job_categories = [x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19]

df_job_cat = pd.concat(job_categories)
df_job_cat

,Occupation,All_workers,All_weekly,M_workers,M_weekly,F_workers,F_weekly
1,MANAGEMENT,12480,1351,7332,1486,5147,1139
32,BUSINESS,5942,1137,2686,1327,3256,1004
61,COMPUTATIONAL,4009,1428,3036,1503,973,1245
78,ENGINEERING,2656,1424,2272,1452,383,1257
100,SCIENCE,1176,1206,662,1379,514,1067
124,SOCIAL SERVICE,2143,889,776,973,1367,845
133,LEGAL,1346,1391,624,1877,722,1135
139,EDUCATION,6884,956,1849,1144,5034,907
151,ARTS,1643,1001,930,1088,713,942
171,HEALTHCARE PROFESSIONAL,6566,1041,1639,1272,4928,991
